In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict
from typing import Literal, Optional
from sklearn.metrics import f1_score
import copy
from matplotlib.ticker import FuncFormatter
from matplotlib.axes import Axes
from matplotlib.figure import Figure
import seaborn.objects as so
import math
import itertools
import matplotlib.ticker as mticker
import random
import os
from torch.nn import RNN
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

# Tiền xử lý dữ liệu

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
dfFilm_C = pd.read_csv("/content/drive/MyDrive/NLP_Đồ án/film_cleaned_ver2.csv") #file sau khi gán thủ công
dfFilm_C.head()

,author,comment,ratings,title,url,cleaned_text,length,has_emoji,label
0,Nguyễn Thanh Trúc,Ta nói nó hài . Bản thân t con gái nhưng rất t...,5/10,Truy Tìm Long Diên Hương,https://www.momo.vn/cinema/truy-tim-long-dien-...,ta nói nó hài bản thân tôi con gái nhưng rất t...,23,0,positive
1,Trần Công Lập,"Hên là coi ở CGV, phim gì mà nhân vật nói chả ...",2/10,Truy Tìm Long Diên Hương,https://www.momo.vn/cinema/truy-tim-long-dien-...,hên coi ở cgv phim gì nhân vật nói chả nghe cá...,51,0,negative
2,Nguyễn Bảo Thoại Trinh,"phim hài, cười banh rạp =)))) Nửa tiếng đầu mì...",7/10,Truy Tìm Long Diên Hương,https://www.momo.vn/cinema/truy-tim-long-dien-...,phim hài cười banh rạp = nửa tiếng đầu mình bi...,36,0,positive
3,Dương Thanh Huy,Nếu bỏ tiền mua vé xem Phim Việt thì tui tốn c...,5/10,Truy Tìm Long Diên Hương,https://www.momo.vn/cinema/truy-tim-long-dien-...,nếu bỏ tiền mua vé xem phim việt tui tốn chưa ...,231,0,positive
4,Trần Thị Thanh Sang,Phim hay và ý nghĩa. Cười rất nhiều. Có nhìu h...,5/10,Truy Tìm Long Diên Hương,https://www.momo.vn/cinema/truy-tim-long-dien-...,phim hay ý nghĩa cười rất nhiều có nhìu hành đ...,14,0,positive


In [6]:
# Lấy ra những cột thật sự cần cho SA
df_needed = dfFilm_C[['cleaned_text', 'label']]
df_needed.head()

,cleaned_text,label
0,ta nói nó hài bản thân tôi con gái nhưng rất t...,positive
1,hên coi ở cgv phim gì nhân vật nói chả nghe cá...,negative
2,phim hài cười banh rạp = nửa tiếng đầu mình bi...,positive
3,nếu bỏ tiền mua vé xem phim việt tui tốn chưa ...,positive
4,phim hay ý nghĩa cười rất nhiều có nhìu hành đ...,positive


In [7]:
df_needed['label'].value_counts()

,count
label,
positive,2005
negative,1841
mixed/neutral,1233
none,328


In [8]:
# Loại bỏ đi Label là none có trong dữ liệu
df_needed = df_needed[df_needed['label'].isin(['positive', 'negative', 'mixed/neutral'])]

# Chuẩn bị dữ liệu cho mô hình học sâu

In [9]:
# Xây dựng Vocabulary cho tập dữ liệu trên
def build_vocab(corpus: pd.Series) -> dict[str, int]:
    """
    Build vocabulary
    """
    # vectorize short_description to get the vocabulary
    vocab_vectorizer = CountVectorizer(min_df=1, max_df=1.0, stop_words=None)
    matrix = vocab_vectorizer.fit_transform(corpus)
    vocab = vocab_vectorizer.vocabulary_

    df = pd.DataFrame(list(vocab.items()), columns=['word', 'index'])
    df['index'] = df['index'] + 2

    df_pad = pd.DataFrame(list({"<PAD>": 0, "<UNK>": 1}.items()), columns=["word", "index"])
    df = pd.concat([df, df_pad], axis=0, ignore_index=True)

    vocab = dict(zip(df['word'], df['index']))

    return vocab

In [10]:
# Tạo dictionary để map label thành số
label = df_needed['label'].unique()
idx = range(len(label))
label_mapping = dict(zip(label, idx))
label_mapping

{'positive': 0, 'negative': 1, 'mixed/neutral': 2}

In [11]:
# Chia tập dữ liệu thành train, val, test dataset
X = df_needed['cleaned_text']
y = df_needed['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.2, stratify=y_train, random_state=42)

In [12]:
vocab=build_vocab(X_train)
len(vocab)

4913

In [13]:
from torch.nn.utils.rnn import pad_sequence
class SentimentDataset(Dataset):
    def __init__(self, vocab, text: pd.Series, label: pd.Series, label_mapping):
        super().__init__()
        self.vocab = vocab
        self.text = text
        self.label = label
        self.label_mapping = label_mapping

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx) -> tuple[torch.LongTensor, torch.LongTensor]:  # iterate through each sample, must have "idx"
        # get text at index
        text_item = self.text.iloc[idx]
        label_item = self.label.iloc[idx]

        # tokenize
        tokens = text_item.split()
        tokens_idx = [self.vocab[token] if token in self.vocab else self.vocab['<UNK>'] for token in tokens] # list
        label_idx = self.label_mapping[label_item]

        return torch.tensor(tokens_idx, dtype=torch.long), torch.tensor(label_idx, dtype=torch.long)

def collate_fn(batch) -> tuple[torch.LongTensor, torch.LongTensor]:
    tokens_idx, label = zip(*batch) # return tuples
    labels = torch.stack(label, dim=0)

    # Pad the sequences
    padded_sequences = pad_sequence(tokens_idx, batch_first=True,
                                    padding_value=vocab["<PAD>"])

    return padded_sequences, labels

In [14]:
train_dataset = SentimentDataset(vocab=vocab, label_mapping=label_mapping,
                                    text=X_train, label=y_train)

val_dataset = SentimentDataset(vocab=vocab, label_mapping=label_mapping,
                                    text=X_val, label=y_val)

test_dataset = SentimentDataset(vocab=vocab, label_mapping=label_mapping,
                                    text=X_test, label=y_test)

In [15]:
train_loader = DataLoader(train_dataset, batch_size = 512, shuffle=True, collate_fn = collate_fn)

val_loader = DataLoader(val_dataset, batch_size = 512, shuffle=False, collate_fn = collate_fn)

test_loader = DataLoader(test_dataset, batch_size = 512, shuffle=False, collate_fn = collate_fn)

# used to create mini-batch

# Xây dựng các mô hình học sâu

In [16]:
# Hàm hỗ trợ
import math
class ModelTraining:
    def __init__(self, model, optimizer, loss_function, device,
                 train_loader, val_loader, test_loader):
        self.model = model
        self.optim = optimizer
        self.loss_fn = loss_function
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.best_state = None

    def calculate_loss_metric_score(self, loader, state: Literal['Train', 'Val', 'Test']):

        total_loss = 0.0
        total_samples = 0
        y_label_pred = []
        y_label_true = []

        for X, y in loader:
            y = y.to(self.device)
            X = X.to(self.device)

            output = self.model(X)

            loss = self.loss_fn(output, y)

            total_loss += loss.item() * len(y)
            total_samples += len(y)

            y_label_true.extend(y.tolist())
            y_label_pred.extend(torch.argmax(output, dim=1).tolist())

            if state == 'Train':
                self.optim.zero_grad()

                loss.backward()

                self.optim.step()

        total_loss /= total_samples

        return y_label_true, y_label_pred, total_loss

    def train_performance(self, metric_fn: callable, kwargs: dict):

        self.model.train()
        y_label_true, y_label_pred, total_loss = self.calculate_loss_metric_score(self.train_loader, 'Train')
        metric_score = metric_fn(y_label_true, y_label_pred, **kwargs)

        return total_loss, metric_score

    def val_performance(self, metric_fn: callable, kwargs: dict):
        self.model.eval()
        with torch.no_grad():
            y_label_true, y_label_pred, total_loss = self.calculate_loss_metric_score(self.val_loader, 'Val')
            metric_score = metric_fn(y_label_true, y_label_pred, **kwargs)

            return total_loss, metric_score

    def training_session(self, epochs: int, metric_fn: callable, kwargs: dict|None = None):
        best_score = float('-inf')

        train_loss_collection = []
        val_loss_collection = []
        train_score_collection = []
        val_score_collection = []

        if kwargs is None:
            kwargs = {}

        for epoch in range(1, epochs + 1):
            train_loss, train_score = self.train_performance(metric_fn, kwargs)
            val_loss, val_score = self.val_performance(metric_fn, kwargs)

            train_loss_collection.append(train_loss)
            val_loss_collection.append(val_loss)

            train_score_collection.append(train_score)
            val_score_collection.append(val_score)

            if val_score > best_score:
                best_score = val_score
                self.best_state = copy.deepcopy(self.model.state_dict())
                print(f"New best score in epoch {epoch}: {val_score}. Saving model parameters...")

            print("=====================")
            print(f"Epoch {epoch} completed")
            print(f"Train Loss: {train_loss:.4f} ; Score: {train_score}")
            print(f"Val Loss: {val_loss:.4f} ; Score: {val_score}")
            print("=====================")

        return train_loss_collection, val_loss_collection, train_score_collection, val_score_collection, self.best_state

    def testing(self, metric_fn: callable, kwargs: dict|None=None):
        if kwargs is None:
            kwargs = {}

        self.model.load_state_dict(self.best_state)

        self.model.eval()
        with torch.no_grad():
            y_label_true, y_label_pred, test_loss = self.calculate_loss_metric_score(self.test_loader, 'Test')
            test_score = metric_fn(y_label_true, y_label_pred, **kwargs)

            print(f"Test Loss: {test_loss} ; Score: {test_score}")

        return test_loss, test_score

In [18]:
class LinePlot:
    def __init__(self, df:pd.DataFrame):
        self.df = df

    @staticmethod
    def readable_numbers(x: float) -> str:
        if x >= 1e6:
            s = '{:1.2f}M'.format(x*1e-6)
        else:
            s = '{:1.2f}K'.format(x*1e-3)
        return s

    def set_locator(self, axes: Axes):
        def is_number(x):
            x = x.replace('−', '-')
            try:
                float(x)
                return True
            except ValueError:
                return False

        x_ticklabels = [tick.get_text() for tick in axes.get_xticklabels()]
        y_ticklabels = [tick.get_text() for tick in axes.get_yticklabels()]

        formatter = FuncFormatter(lambda val, pos: self.readable_numbers(val))

        if is_number(x_ticklabels[0]) == True:
            axes.xaxis.set_major_formatter(formatter)
        if is_number(y_ticklabels[0]) == True:
            axes.yaxis.set_major_formatter(formatter)

    def single_plot(
        self,
        x_axis: str,
        y_axis: str,
        hue: str | None = None,
        style: str | None = None,
        size: str | None = None,
        kwarg: dict | None = None,
        readable_label: bool = False,
        ax: Axes = None
    ) -> Axes:

        if ax is None:
            ax = plt.gca()
            fig = ax.get_figure()
            fig.set_size_inches(10, 6)
        if kwarg is None:
            kwarg = {}

        # --- 🎨 Line Plot ---
        line = sns.lineplot(
            data=self.df,
            x=x_axis,
            y=y_axis,
            hue=hue,
            style=style,
            size=size,
            ax=ax,
            # marker="o",
            linewidth=2.2,
            # markersize=4,
            alpha=0.9,
            palette="viridis",
            **kwarg
        )


        ax.set_title(
            f"{y_axis} vs {x_axis}",
            fontsize=12,
            fontweight="regular",
            # fontname="Times New Roman",
            # pad=15
        )
        ax.set_xlabel(x_axis, fontsize=12, labelpad=5,) # fontname="Times New Roman"
        ax.set_ylabel(y_axis, fontsize=12, labelpad=10,) # fontname="Times New Roman"

        ax.tick_params(axis="x", rotation=0, labelsize=11)
        ax.tick_params(axis="y", rotation=0, labelsize=11)
        ax.grid(True, which="major", linestyle="--", linewidth=0.7, alpha=0.5)

        if hue or style or size:
            leg = ax.legend(
                title="Legend",
                title_fontsize=11,
                fontsize=10,
                loc="best",
                frameon=False,
                fancybox=True,
                framealpha=0.9,
            )
            leg.get_frame().set_linewidth(0.7)
            leg.get_frame().set_edgecolor("gray")


        if readable_label:
            self.set_locator(ax)


        sns.despine(ax=ax, top=True, right=True)

        return line

    def multiple_plots(self, x_axis_list: list[str], y_axis_list: list[str], hue_list: list[str|None],
                       kwarg: dict, plots_each_row: int, size_list: list[str|None], style_list: list[str|None],
                       readable_label: bool = False) -> tuple[Figure, np.array[Axes]]:


        combination = list(zip(x_axis_list, y_axis_list, hue_list, size_list, style_list))

        length = len(combination)
        number_rows = math.ceil(length / plots_each_row)

        fig, axes = plt.subplots(
    nrows=number_rows,
    ncols=plots_each_row,
    figsize=(16, 10),
    squeeze=False   # ⭐ quan trọng
)


        coordinates = list(itertools.product(range(number_rows), range(plots_each_row)))

        for coor, com in zip(coordinates, combination):
            x, y, hue, size, style = com
            row, col = coor
            ax = axes[row, col]
            self.single_plot(
                x_axis=x,
                y_axis=y,
                hue=hue,
                size=size,
                style=style,
                readable_label=readable_label,
                kwarg=kwarg,
                ax=ax
            )

        # hide any unused axes
        for coor in coordinates[length:]:
            fig.delaxes(axes[coor])

        return fig, axes

TypeError: 'builtin_function_or_method' object is not subscriptable

In [19]:
def multiple_plots(
    self,
    x_axis_list: list[str],
    y_axis_list: list[str],
    hue_list: list[str | None],
    kwarg: dict,
    plots_each_row: int,
    size_list: list[str | None],
    style_list: list[str | None],
    readable_label: bool = False
) -> tuple[Figure, np.ndarray]:

    combination = list(zip(x_axis_list, y_axis_list, hue_list, size_list, style_list))
    length = len(combination)
    number_rows = math.ceil(length / plots_each_row)

    fig, axes = plt.subplots(
        nrows=number_rows,
        ncols=plots_each_row,
        figsize=(16, 10),
        squeeze=False
    )

    coordinates = list(itertools.product(range(number_rows), range(plots_each_row)))

    for (row, col), (x, y, hue, size, style) in zip(coordinates, combination):
        ax = axes[row, col]
        self.single_plot(
            x_axis=x,
            y_axis=y,
            hue=hue,
            size=size,
            style=style,
            readable_label=readable_label,
            kwarg=kwarg,
            ax=ax
        )

    # hide unused axes
    for row, col in coordinates[length:]:
        fig.delaxes(axes[row, col])

    return fig, axes

## Mô hình RNN

### Xây dựng mô hình

In [20]:
class RNNModel(nn.Module):
    def __init__(self, vocab: dict[str, int], emb_dim: int, hidden_size: int, num_layers:int,
                 num_classes: int, pooling_fn: Literal['mean', 'sum', 'max']|None = None):
        super().__init__()
        self.vocab = vocab
        self.vocab_size = len(vocab)
        self.emb_dim = emb_dim
        self.pooling_fn = pooling_fn
        self.embedding = nn.Embedding(self.vocab_size, emb_dim, padding_idx=vocab['<PAD>'])
        self.dropout_emb = nn.Dropout(p=0.35)
        self.rnn = nn.RNN(input_size=emb_dim, hidden_size=hidden_size, num_layers=num_layers,
                         nonlinearity='tanh', bias=True, batch_first=True, dropout=0, bidirectional=False)

        self.mlp = nn.Sequential(OrderedDict([
            ("fc1", nn.Linear(hidden_size, 64)),
            ("ln1", nn.LayerNorm(64)),
            ("relu1", nn.LeakyReLU()),
            ("drop1", nn.Dropout(p=0.35)),

            ("fc2", nn.Linear(64, 128)),
            ("ln2", nn.LayerNorm(128)),
            ("relu2", nn.LeakyReLU()),
            ("drop2", nn.Dropout(p=0.35)),

            ("fc3", nn.Linear(128, 128)),
            ("ln3", nn.LayerNorm(128)),
            ("relu3", nn.LeakyReLU()),
            ("drop3", nn.Dropout(p=0.35)),

            ('out', nn.Linear(128, num_classes))
        ]))

    def forward(self, tokens_idx: torch.Tensor):
        tokens_emb = self.embedding(tokens_idx)
        tokens_emb = self.dropout_emb(tokens_emb)

        mask = (tokens_idx != self.vocab['<PAD>']).float().unsqueeze(-1)

        lengths = mask.sum(dim=1).long()
        lengths = lengths.squeeze()

        packed = pack_padded_sequence(tokens_emb, lengths.cpu(),
                                      batch_first = True, enforce_sorted = False)

        output_packed, last_hidden_state = self.rnn(packed)

        output, out_lengths = pad_packed_sequence(
            output_packed,
            batch_first=True
        )

        out_lengths = out_lengths.to(output.device)

        if self.pooling_fn == 'mean':
            final_hidden_state = output.sum(dim=1).squeeze() / lengths.unsqueeze(-1)
        elif self.pooling_fn == 'sum':
            final_hidden_state = output.sum(dim=1).squeeze()
        elif self.pooling_fn == 'max':
            B, num_tokens, hidden_size = output.shape
            device = output.device

            non_pad_tokens = torch.arange(num_tokens, device=device).unsqueeze(0)
            mask_for_max_pooling = (non_pad_tokens < out_lengths.unsqueeze(1))
            mask_for_max_pooling = mask_for_max_pooling.unsqueeze(-1)

            output_mask = output.masked_fill(~mask_for_max_pooling, float('-inf'))
            final_hidden_state, max_value_index = output.max(dim=1)
        else:
            final_hidden_state = last_hidden_state.transpose(0, 1)
            final_hidden_state = final_hidden_state.squeeze()

        output_mlp = self.mlp(final_hidden_state)

        return output_mlp

### Huấn luyện mô hình

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 50

loss_function = nn.CrossEntropyLoss()
metric_fn = f1_score

rnn_params = {'vocab': vocab,
              'emb_dim': 128,
              'num_classes': y_train.nunique(),
              'hidden_size': 128,
              'num_layers': 1,
              'pooling_fn': 'max'
             }

In [22]:
model = RNNModel(**rnn_params)
model = torch.nn.DataParallel(model)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr= 0.003, weight_decay=1e-2)

performance = ModelTraining(model=model, optimizer=optimizer, loss_function=loss_function,
                             device=device, train_loader=train_loader, val_loader=val_loader,
                             test_loader=test_loader)

In [23]:
train_loss_collection, val_loss_collection, train_score_collection, val_score_collection, best_state = \
performance.training_session(epochs=epochs, metric_fn=metric_fn, kwargs={'average': 'macro'})

New best score in epoch 1: 0.2145734863411588. Saving model parameters...
Epoch 1 completed
Train Loss: 1.1699 ; Score: 0.34568007878691587
Val Loss: 1.0780 ; Score: 0.2145734863411588
New best score in epoch 2: 0.27477167394071966. Saving model parameters...
Epoch 2 completed
Train Loss: 1.1044 ; Score: 0.30853061111748287
Val Loss: 1.0734 ; Score: 0.27477167394071966
Epoch 3 completed
Train Loss: 1.0810 ; Score: 0.31150536555420993
Val Loss: 1.0618 ; Score: 0.2589981236140331
New best score in epoch 4: 0.43996966782755864. Saving model parameters...
Epoch 4 completed
Train Loss: 1.0681 ; Score: 0.32010102159241227
Val Loss: 0.9845 ; Score: 0.43996966782755864
New best score in epoch 5: 0.47183635391424056. Saving model parameters...
Epoch 5 completed
Train Loss: 0.9524 ; Score: 0.4328203896114126
Val Loss: 0.8928 ; Score: 0.47183635391424056
New best score in epoch 6: 0.5850628983056124. Saving model parameters...
Epoch 6 completed
Train Loss: 0.8444 ; Score: 0.5447663819749108
Val L

In [ ]:
df_train = pd.DataFrame({'loss': train_loss_collection,
                        'score': train_score_collection,
                        'type': "train"})

df_val = pd.DataFrame({'loss': val_loss_collection,
                        'score': val_score_collection,
                        'type': "val"})
df = pd.concat((df_train, df_val), axis=0).reset_index()
df['index'] = df['index'] + 1

In [ ]:
X = ['index', 'index']
y = ['loss', 'score']
hue = ['type', 'type']
style_list = [None for _ in range(2)]
size_list = [None for _ in range(2)]

plotter = LinePlot(df)
fig, axes = plotter.multiple_plots(x_axis_list=X, y_axis_list=y, hue_list=hue,
                                      style_list=style_list, size_list=size_list, plots_each_row=2,
                                      kwarg={}, readable_label=False)

In [ ]:
axes[0,0].set_title('Loss Graph')
axes[0,0].set_xlabel('Epoch')

axes[0,1].set_title('Val Graph')
axes[0,1].set_xlabel('Epoch')

fig

In [ ]:
test_loss_rnn, test_score_rnn = performance.testing(metric_fn=metric_fn, kwargs={'average': 'macro'})

In [ ]:
best_path = "best_rnn_model_5.pth"

torch.save({
    "model_state": best_state,
    "model_class": model.__class__.__name__,
    "model_args": rnn_params,
    "optimizer_state": optimizer.state_dict(),
    "epochs": epochs
}, best_path)

## Mô Hình LSTM

### Xây dựng mô hình

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, vocab: dict[str, int], emb_dim: int, hidden_size: int, num_layers:int,
                 num_classes: int, pooling_fn: Literal['mean', 'sum', 'max']|None = None):
        super().__init__()
        self.vocab = vocab
        self.vocab_size = len(vocab)
        self.emb_dim = emb_dim
        self.pooling_fn = pooling_fn
        self.embedding = nn.Embedding(self.vocab_size, emb_dim, padding_idx=vocab['<PAD>'])
        self.dropout_emb = nn.Dropout(p=0.35)

        self.lstm = nn.LSTM(input_size=emb_dim, hidden_size=hidden_size, num_layers=num_layers,
                           bias=True, batch_first=True, dropout=0, bidirectional=False, proj_size=0)

        self.mlp = nn.Sequential(OrderedDict([
            ("fc1", nn.Linear(hidden_size, 64)),
            ("ln1", nn.LayerNorm(64)),
            ("relu1", nn.LeakyReLU()),
            ("drop1", nn.Dropout(p=0.35)),

            ("fc2", nn.Linear(64, 128)),
            ("ln2", nn.LayerNorm(128)),
            ("relu2", nn.LeakyReLU()),
            ("drop2", nn.Dropout(p=0.25)),

            ("fc3", nn.Linear(128, 128)),
            ("ln3", nn.LayerNorm(128)),
            ("relu3", nn.LeakyReLU()),
            ("drop3", nn.Dropout(p=0.35)),

            ('out', nn.Linear(128, num_classes))
        ]))

    def forward(self, tokens_idx: torch.Tensor):
        tokens_emb = self.embedding(tokens_idx)

        tokens_emb = self.dropout_emb(tokens_emb)

        mask = (tokens_idx != self.vocab['<PAD>']).float().unsqueeze(-1)

        lengths = mask.sum(dim=1).long()
        lengths = lengths.squeeze(-1)

        packed = pack_padded_sequence(tokens_emb, lengths.cpu(),
                                      batch_first = True, enforce_sorted = False)

        output_packed, (last_hidden_state, last_cell_state) = self.lstm(packed)

        output, out_lengths = pad_packed_sequence(
            output_packed,
            batch_first=True
        )

        out_lengths = out_lengths.to(output.device)

        if self.pooling_fn == 'mean':
            final_hidden_state = output.sum(dim=1).squeeze() / lengths.unsqueeze(-1)
        elif self.pooling_fn == 'sum':
            final_hidden_state = output.sum(dim=1).squeeze()
        elif self.pooling_fn == 'max':
            B, num_tokens, hidden_size = output.shape
            device = output.device

            non_pad_tokens = torch.arange(num_tokens, device=device).unsqueeze(0)
            mask_for_max_pooling = (non_pad_tokens < out_lengths.unsqueeze(1))
            mask_for_max_pooling = mask_for_max_pooling.unsqueeze(-1)

            output_mask = output.masked_fill(~mask_for_max_pooling, float('-inf'))
            final_hidden_state, max_value_index = output.max(dim=1)
        else:
            final_hidden_state = last_hidden_state.transpose(0, 1)
            final_hidden_state = final_hidden_state.squeeze()

        output_mlp = self.mlp(final_hidden_state)

        return output_mlp

### Huấn luyện mô hình

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 50

loss_function = nn.CrossEntropyLoss()
metric_fn = f1_score

lstm_params = {'vocab': vocab,
              'emb_dim': 128,
              'num_classes': y_train.nunique(),
              'hidden_size': 128,
              'num_layers': 1,
              'pooling_fn': 'max'
             }

In [ ]:
model = LSTMModel(**lstm_params)
model = torch.nn.DataParallel(model)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr= 0.005, weight_decay=1e-2)

performance = ModelTraining(model=model, optimizer=optimizer, loss_function=loss_function,
                             device=device, train_loader=train_loader, val_loader=val_loader,
                             test_loader=test_loader)

In [ ]:
train_loss_collection, val_loss_collection, train_score_collection, val_score_collection, best_state = \
performance.training_session(epochs=epochs, metric_fn=metric_fn, kwargs={'average': 'macro'})

In [ ]:
df_train = pd.DataFrame({'loss': train_loss_collection,
                        'score': train_score_collection,
                        'type': "train"})

df_val = pd.DataFrame({'loss': val_loss_collection,
                        'score': val_score_collection,
                        'type': "val"})
df = pd.concat((df_train, df_val), axis=0).reset_index()
df['index'] = df['index'] + 1

In [ ]:
X = ['index', 'index']
y = ['loss', 'score']
hue = ['type', 'type']
style_list = [None for _ in range(2)]
size_list = [None for _ in range(2)]

plotter = LinePlot(df)
fig, axes = plotter.multiple_plots(x_axis_list=X, y_axis_list=y, hue_list=hue,
                                      style_list=style_list, size_list=size_list, plots_each_row=2,
                                      kwarg={}, readable_label=False)

In [ ]:
axes[0,0].set_title('Loss Graph')
axes[0,0].set_xlabel('Epoch')

axes[0,1].set_title('Val Graph')
axes[0,1].set_xlabel('Epoch')

fig

In [ ]:
test_loss_lstm, test_score_lstm = performance.testing(metric_fn=metric_fn, kwargs={'average': 'macro'})

In [ ]:
import os
best_path = "best_lstm_model_3.pth"

torch.save({
    "model_state": best_state,
    "model_class": model.__class__.__name__,
    "model_args": lstm_params,
    "optimizer_state": optimizer.state_dict(),
    "epochs": epochs
}, best_path)

## Mô hình GRU

### Xây dựng mô hình

In [ ]:
from torch.nn import RNN
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

class GRUModel(nn.Module):
    def __init__(self, vocab: dict[str, int], emb_dim: int, hidden_size: int, num_layers:int,
                 num_classes: int, pooling_fn: Literal['mean', 'sum', 'max']|None = None):
        super().__init__()
        self.vocab = vocab
        self.vocab_size = len(vocab)
        self.emb_dim = emb_dim
        self.pooling_fn = pooling_fn
        self.embedding = nn.Embedding(self.vocab_size, emb_dim, padding_idx=vocab['<PAD>'])
        self.dropout_emb = nn.Dropout(p=0.35)

        self.gru = nn.GRU(input_size=emb_dim, hidden_size=hidden_size, num_layers=num_layers,
                           bias=True, batch_first=True, dropout=0, bidirectional=False)

        self.mlp = nn.Sequential(OrderedDict([
            ("fc1", nn.Linear(hidden_size, 64)),
            ("ln1", nn.LayerNorm(64)),
            ("relu1", nn.LeakyReLU()),
            ("drop1", nn.Dropout(p=0.35)),

            ("fc2", nn.Linear(64, 128)),
            ("ln2", nn.LayerNorm(128)),
            ("relu2", nn.LeakyReLU()),
            ("drop2", nn.Dropout(p=0.3)),

            ("fc3", nn.Linear(128, 128)),
            ("ln3", nn.LayerNorm(128)),
            ("relu3", nn.LeakyReLU()),
            ("drop3", nn.Dropout(p=0.35)),

            ('out', nn.Linear(128, num_classes))
        ]))

    def forward(self, tokens_idx: torch.Tensor):
        tokens_emb = self.embedding(tokens_idx)

        tokens_emb = self.dropout_emb(tokens_emb)

        mask = (tokens_idx != self.vocab['<PAD>']).float().unsqueeze(-1)

        lengths = mask.sum(dim=1).long()
        lengths = lengths.squeeze(-1)

        packed = pack_padded_sequence(tokens_emb, lengths.cpu(),
                                      batch_first = True, enforce_sorted = False)


        output_packed, last_hidden_state = self.gru(packed)

        output, out_lengths = pad_packed_sequence(
            output_packed,
            batch_first=True
        )

        out_lengths = out_lengths.to(output.device)

        if self.pooling_fn == 'mean':
            final_hidden_state = output.sum(dim=1).squeeze() / lengths.unsqueeze(-1)
        elif self.pooling_fn == 'sum':
            final_hidden_state = output.sum(dim=1).squeeze()
        elif self.pooling_fn == 'max':
            B, num_tokens, hidden_size = output.shape
            device = output.device


            non_pad_tokens = torch.arange(num_tokens, device=device).unsqueeze(0)
            mask_for_max_pooling = (non_pad_tokens < out_lengths.unsqueeze(1))
            mask_for_max_pooling = mask_for_max_pooling.unsqueeze(-1)

            output_mask = output.masked_fill(~mask_for_max_pooling, float('-inf'))
            final_hidden_state, max_value_index = output.max(dim=1)
        else:
            final_hidden_state = last_hidden_state.transpose(0, 1)
            final_hidden_state = final_hidden_state.squeeze()

        output_mlp = self.mlp(final_hidden_state)

        return output_mlp

### Huấn luyện mô hình

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 50

loss_function = nn.CrossEntropyLoss()
metric_fn = f1_score

gru_params = {'vocab': vocab,
              'emb_dim': 128,
              'num_classes': y_train.nunique(),
              'hidden_size': 128,
              'num_layers': 1,
              'pooling_fn': 'max'
             }

In [ ]:
model = GRUModel(**gru_params)
model = torch.nn.DataParallel(model)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr= 0.005, weight_decay=1e-2)

performance = ModelTraining(model=model, optimizer=optimizer, loss_function=loss_function,
                             device=device, train_loader=train_loader, val_loader=val_loader,
                             test_loader=test_loader)

In [ ]:
train_loss_collection, val_loss_collection, train_score_collection, val_score_collection, best_state = \
performance.training_session(epochs=epochs, metric_fn=metric_fn, kwargs={'average': 'macro'})

In [ ]:
df_train = pd.DataFrame({'loss': train_loss_collection,
                        'score': train_score_collection,
                        'type': "train"})

df_val = pd.DataFrame({'loss': val_loss_collection,
                        'score': val_score_collection,
                        'type': "val"})
df = pd.concat((df_train, df_val), axis=0).reset_index()
df['index'] = df['index'] + 1

In [ ]:
X = ['index', 'index']
y = ['loss', 'score']
hue = ['type', 'type']
style_list = [None for _ in range(2)]
size_list = [None for _ in range(2)]

plotter = LinePlot(df)
fig, axes = plotter.multiple_plots(x_axis_list=X, y_axis_list=y, hue_list=hue,
                                      style_list=style_list, size_list=size_list, plots_each_row=2,
                                      kwarg={}, readable_label=False)

In [ ]:
axes[0,0].set_title('Loss Graph')
axes[0,0].set_xlabel('Epoch')

axes[0,1].set_title('Val Graph')
axes[0,1].set_xlabel('Epoch')

fig

In [ ]:
test_loss_gru, test_score_gru = performance.testing(metric_fn=metric_fn, kwargs={'average': 'macro'})

In [ ]:
import os
best_path = "best_gru_model.pth"

torch.save({
    "model_state": best_state,
    "model_class": model.__class__.__name__,
    "model_args": gru_params,
    "optimizer_state": optimizer.state_dict(),
    "epochs": epochs
}, best_path)

# So sánh hiệu quả giữa các mô hình

In [ ]:
comparison = pd.DataFrame({'Model': ['RNN', 'LSTM', 'GRU'],
                          'Loss': [test_loss_rnn, test_loss_lstm, test_loss_gru],
                          'Score': [test_score_rnn, test_score_lstm, test_score_gru]})
comparison